## Data Ingestion Pipeline

## Document Parsing

In [ ]:
import sys
!{sys.executable} -m pip install langchain langchain-core langchain-community pypdf pymupdf sentence-transformers faiss-cpu chromadb langchain-groq python-dotenv typesense langchain_openai langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 5.4 MB/s eta 

In [ ]:
# Document Structure
from langchain_core.documents import Document

In [ ]:
doc = Document(
    page_content="Machine learning is fun",
    metadata = {                                                      # to apply filters
        "source" : "example.txt",
        "page_no" : 1,
        "author" : "firdous",
        "date_created" : "2025-01-01",
    }
)
doc

Document(metadata={'source': 'example.txt', 'page_no': 1, 'author': 'firdous', 'date_created': '2025-01-01'}, page_content='Machine learning is fun')

In [ ]:
# create simple txt file
import os
os.makedirs('data/text_files',exist_ok=True)

In [ ]:
sample_texts={
    "data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",

    "data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems


    """
}

for filepath,content in sample_texts.items():
  with open(filepath,'w',encoding="utf-8") as f:
    f.write(content)
print("Sample text files created")

Sample text files created


In [ ]:
# TextLoader
from langchain_community.document_loaders import TextLoader
loader = TextLoader('data/text_files/python_intro.txt')
document = loader.load()
print(document)

[Document(metadata={'source': 'data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.')]


In [ ]:
# DirectoryLoader
from langchain_community.document_loaders import DirectoryLoader

dir_loader = DirectoryLoader(
    'data/text_files',
    glob = "**/*.txt",
    loader_cls = TextLoader,
    loader_kwargs={'encoding' : 'utf-8'},
    show_progress = True
)

documents = dir_loader.load()
documents

100%|██████████| 2/2 [00:00<00:00, 2069.73it/s]


[Document(metadata={'source': 'data/text_files/machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learning: Learning with labeled data\n2. Unsupervised Learning: Finding patterns in unlabeled data\n3. Reinforcement Learning: Learning through rewards and penalties\n\nApplications include image recognition, speech processing, and recommendation systems\n\n\n    '),
 Document(metadata={'source': 'data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogram

In [ ]:
#
from langchain_community.document_loaders import PyMuPDFLoader,PyPDFLoader #PyMuPDF better to PyPDF

dir_loader = DirectoryLoader(
    'data/pdf_files',
    glob = "**/*.pdf",
    loader_cls = PyMuPDFLoader,
    show_progress=True
)

documents = dir_loader.load()
documents
# Visit langchain documentation for other types of documents loader

100%|██████████| 4/4 [00:09<00:00,  2.40s/it]


[Document(metadata={'producer': '', 'creator': '', 'creationdate': '2014-07-07T12:20:41+00:00', 'source': 'data/pdf_files/making_sense_of_data_i_a_practical_guide_to_exploratory_data_analysis_and_data_mining-myatt_etal-wiley-2014.pdf', 'file_path': 'data/pdf_files/making_sense_of_data_i_a_practical_guide_to_exploratory_data_analysis_and_data_mining-myatt_etal-wiley-2014.pdf', 'total_pages': 250, 'format': 'PDF 1.7', 'title': 'Making Sense of Data I', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2014-07-09T16:01:25+02:00', 'trapped': '', 'modDate': "D:20140709160125+02'00'", 'creationDate': 'D:20140707122041Z', 'page': 0}, page_content=''),
 Document(metadata={'producer': '', 'creator': '', 'creationdate': '2014-07-07T12:20:41+00:00', 'source': 'data/pdf_files/making_sense_of_data_i_a_practical_guide_to_exploratory_data_analysis_and_data_mining-myatt_etal-wiley-2014.pdf', 'file_path': 'data/pdf_files/making_sense_of_data_i_a_practical_guide_to_exploratory_data_analysis_and_d

## RAG Pipeline : Data Ingestion to vector DB pipeline

### Data Ingestion

In [ ]:
import os
from langchain_community.document_loaders import  PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

In [ ]:
# Read all the pdfs inside directory
def process_all_pdfs(pdf_directory):
  """Process all pdfs in a directory"""

  all_documents = []
  pdf_dir = Path(pdf_directory)

  pdf_files = list(pdf_dir.glob("**/*.pdf"))
  print(f"Found {len(pdf_files)} PDF files to process")

  for pdf_file in pdf_files:
    print(f'\nProcessing : {pdf_file.name}')
    try:
      loader = PyMuPDFLoader(str(pdf_file))
      documents = loader.load()

      for doc in documents:
        doc.metadata['source_file'] = pdf_file.name
        doc.metadata['file_type'] = 'pdf'

      all_documents.extend(documents)
      print(f"Loaded {len(documents)} pages")
    except Exception as e:
      print(f"Error : {e}")

  print(f"Total documents loaded: {len(all_documents)}")
  return all_documents


In [ ]:
all_pdf_documents = process_all_pdfs('data/pdf_files')

Found 4 PDF files to process

Processing : making_sense_of_data_i_a_practical_guide_to_exploratory_data_analysis_and_data_mining-myatt_etal-wiley-2014.pdf
Loaded 250 pages

Processing : introduction_to_probability_and_statistics_for_engineers_and_scientists-ross-elsevier-2020.pdf
Loaded 692 pages

Processing : learning_statistics_with_r_a_tutorial_for_psychology_students_and_other_beginners-navarro--2013.pdf
Loaded 542 pages

Processing : ISLRv2_corrected_June_2023.pdf
Loaded 615 pages
Total documents loaded: 2099


### Chunking

In [ ]:
# Text Splitting get into chuncks
def split_documents(documents,chunk_size=1000,chunk_overlap=200):
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size = chunk_size,
      chunk_overlap = chunk_overlap,
      length_function = len,
      separators = ["\n\n","\n"," ",""]
  )

  split_docs = text_splitter.split_documents(documents)
  print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

  if split_docs:
    print(f"\nExample chunk")
    print(f"Content: {split_docs[0].page_content[:200]}")
    print(f"Metadata: {split_docs[0].metadata}")

  return split_docs


In [ ]:
chunks = split_documents(all_pdf_documents)

Split 2099 documents into 6034 chunks

Example chunk
Content: MAKING SENSE OF
DATA I
Metadata: {'producer': '', 'creator': '', 'creationdate': '2014-07-07T12:20:41+00:00', 'source': 'data/pdf_files/making_sense_of_data_i_a_practical_guide_to_exploratory_data_analysis_and_data_mining-myatt_etal-wiley-2014.pdf', 'file_path': 'data/pdf_files/making_sense_of_data_i_a_practical_guide_to_exploratory_data_analysis_and_data_mining-myatt_etal-wiley-2014.pdf', 'total_pages': 250, 'format': 'PDF 1.7', 'title': 'Making Sense of Data I', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2014-07-09T16:01:25+02:00', 'trapped': '', 'modDate': "D:20140709160125+02'00'", 'creationDate': 'D:20140707122041Z', 'page': 2, 'source_file': 'making_sense_of_data_i_a_practical_guide_to_exploratory_data_analysis_and_data_mining-myatt_etal-wiley-2014.pdf', 'file_type': 'pdf'}


### Embedding

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid                                               # for id of records in vector DB
from typing import List,Dict,Any,Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
class EmbeddingManager():
  """Handles document generation using SentenceTransformer"""

  def __init__(self,model_name : str = "all-MiniLM-L6-v2"):
    self.model_name = model_name
    self.model = None
    self._load_model()

  def _load_model(self):
    """"Load the SentenceTransformer model"""
    try:
      print(f"Loading embedding model: {self.model_name}")
      self.model = SentenceTransformer(self.model_name)
      print(f"Model loaded successfully.Embedding dimensions : {self.model.get_sentence_embedding_dimension()}")
    except Exception as e:
      print(f"Error loading model {self.model_name} : {e}")
      raise

  def generate_embeddings(self,texts:List[str]) -> np.ndarray:
    if not self.model:
      raise ValueError("Model not loaded")

    print(f"Generating embeddings for {len(texts)} texts..")
    embeddings = self.model.encode(texts,show_progress_bar=True)
    print(f"Generated embeddings with shape : {embeddings.shape}")

    return embeddings




In [ ]:
embedding_manager = EmbeddingManager()

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully.Embedding dimensions : 384


### VectorDB

In [ ]:
class VectorStore():

  def __init__(self,collection_name : str = 'pdf_documents',persist_directory : str = 'data/vector_store'):
    self.collection_name = collection_name
    self.persist_directory = persist_directory
    self.client = None
    self.collection = None
    self._initialize_store()

  def _initialize_store(self):
    """"Initialize ChromaDB and collection"""

    try:
      os.makedirs(self.persis_directory,exist_ok = True)
      # Create persistent ChromaDB clinet
      self.client = chromdb.PersistentClient(path = self.persist_directory)

      # Get or create collection
      self.collections = self.client.get_or_create_collection(
          name = self.collection_name,
          metadata = {'description':'PDF document embeddings for RAG'}
      )

      print(f"Vector store initialized. Collection: {self.collection_name}")
      print(f"Existing documents in collectio: {self.collection.count()}")
    except Exception as e:
      print(f"Error initializing vector store : {e}")

  def add_documents(self,documents : List[Any],embeddings:np.ndarray):

    if(len(documents) != len(embeddings)):
      raise ValueError("Number of documents must match number of embeddings")

    print(f"Adding {len(documents)} documents to vector store")

    # Prepare data for chromadb
    ids = []
    metadatas = []
    documents_text = []
    embeddings_list = []

    for i ,(doc,embedding) in enumerate(zip(documents,embeddings)):
      doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
      ids.append(doc_id)

      # Prepare metadata
      metadata = dict{doc.metadata}
      metadata['doc_index'] = i
      metadata['content_length'] = len(doc.page_content)
      metadatas.append(metadata)

      # Document content
      documents_text.append(doc.page_content)

      # Embedding
      embeddings_list.append(embedding.tolist())

      try:
        self.collection.add(
            ids=ids,
            embeddings=embeddings_list,
            metadatas = metadatas,
            documents= documents_text
        )

        print(f"Sucessfully added {len(documents)} documents to vector store")
        print(f"Total documents in collection: {self.collection.count()}")
      except Exception as e:
        print(f"Error adding documents to vector store : {e}")

SyntaxError: invalid syntax (ipython-input-1414261166.py, line 47)